Francisco Javier Piqueras Martínez

# TP3 - Apartado 2 - Infraestructuras Computacionales para el Procesamiento de Datos Masivos
## Ejercicio 2

Desarrollar un notebook de Jupyter, denominado “hashtags.ipynb”, en el que se utilice como fuente de datos Kafka, y en concreto el topic kafkaTwitter. La duración del batch será de 5 segundos. Se procesarán los tweets que lleguen para extraer los hashtags que contengan (tener en cuenta que todos los hashtags comienzan por el carácter ‘#’). Se irán mostrando, cada vez que se procese el batch (5 segundos) los diez hashtags más utilizados desde el inicio del programa hasta ese momento y el número total de apariciones de cada uno, ordenados de mayor a menor frecuencia.

Realizamos los imports necesarios:

In [1]:
import findspark
findspark.init('/Users/javierpiquerasmartinez/dev/spark-2.4.4-bin-hadoop2.7')

In [2]:
import os
os.environ['JAVA_HOME']='/Library/Java/JavaVirtualMachines/jdk1.8.0_231.jdk/Contents/Home/jre'
os.environ['PYSPARK_SUBMIT_ARGS']='--master local[*] --packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.4 pyspark-shell'

In [3]:
import pyspark
import pyspark.streaming
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

Creamos la función para, más adelante, devolver los campos que sigan cierto patrón, o null en caso contrario.

In [4]:
import re

def getRegex(word, regex):
    match_obj = re.search(regex, word)
    
    if type(match_obj) is None:
        return "NULL"
    else:
        return match_obj.group()

Inicializamos el SparkContext y el StreamingContext, con una duración de batch de 5 segundos

In [5]:
sc = SparkContext(master="local[*]", appName="Hashtags")
ssc = StreamingContext(sc, batchDuration=5)
ssc.checkpoint("checkpoint")

Para la recepción de datos, creamos un DirectStream que atienda a el evento 'kafkaTwitter' en el servidor localhost:9092.

In [6]:
dstream = KafkaUtils.createDirectStream(
    ssc, 
    topics = ['kafkaTwitter'],
    kafkaParams = {
        'bootstrap.servers' : 'localhost:9092'
    })

Seguidamente, recogemos todos los tweets y realizamos un filtrado de solamente aquellas palabras que contengan delante el símbolo '#'.

In [7]:
hashtags = dstream.flatMap(
    lambda tweet: (tweet[1].split(" "))
).filter(
    lambda word: ("#" in word)
)

Y comprobamos que sigan el patrón '#\w+'

In [8]:
real_hashtags = hashtags.map(lambda x: (getRegex(x,'#\w+')))

Seguidamente, creamos una tupla de (hashtag, 1)

In [12]:
num_hashtags_batch = real_hashtags.map( lambda ht: (ht, 1))

In [13]:
def updateNum(values, count):
    return sum(values) + (count or 0)

Y la reducimos aquí para sumar cada aparcición de un mismo hashtag:

In [14]:
num_hashtags = num_hashtags_batch.updateStateByKey(updateFunc=updateNum)

In [15]:
result = num_hashtags.transform( 
    lambda ht_count: (ht_count.sortBy(
        lambda pair: ( pair[1]), ascending=False
    ))
)

Finalmente, imprimimos el resultado:

In [16]:
result.pprint(10)

In [17]:
ssc.start()

-------------------------------------------
Time: 2020-01-12 18:10:20
-------------------------------------------
('#Apple', 6)
('#UAM', 3)
('#FelizLunes', 3)
('#ForoCambio11J', 3)
('#UnasPrimariasParaGanar', 3)
('#OSX', 3)
('#YoHagoHistoria', 2)
('#cumPPlimos', 2)
('#GobernarParaLaMayoria', 2)
('#UniDeVerano', 2)
...

-------------------------------------------
Time: 2020-01-12 18:10:25
-------------------------------------------
('#Apple', 6)
('#UAM', 3)
('#FelizLunes', 3)
('#ForoCambio11J', 3)
('#UnasPrimariasParaGanar', 3)
('#OSX', 3)
('#GobernarParaLaMayoria', 3)
('#YoHagoHistoria', 2)
('#cumPPlimos', 2)
('#UniDeVerano', 2)
...

-------------------------------------------
Time: 2020-01-12 18:10:30
-------------------------------------------
('#Apple', 6)
('#UAM', 3)
('#FelizLunes', 3)
('#ForoCambio11J', 3)
('#UnasPrimariasParaGanar', 3)
('#OSX', 3)
('#GobernarParaLaMayoria', 3)
('#YoHagoHistoria', 2)
('#cumPPlimos', 2)
('#UniDeVerano', 2)
...

-------------------------------------